## Important Imports

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt



In [ ]:
from test_functions import idx_gen #index_generator
from test_functions import lable_loading #load_labels
from test_functions import con_data #data_constructor
from test_functions import data_frame_generator#dataframe_generator
from test_functions import idx_extr#index_extractor
from test_functions import c_matrix # plot_confusion_matrix
from test_functions import labels_updater #phase1_labels_updater

## Setting up parameters and directories

In [ ]:
image_height = 128
image_width = 128
DIM = (128,128)
test_data_set= 5000
SEED=12 
label_file_test = "/home/pramod/COVID-19_Detection/image/archive/test_COVIDx_CT-2A.txt"

## Loading test data

In [ ]:
fnames_test , classes_test , bboxes_test = lable_loading(label_file_test)
test_index = idx_gen(fnames_test , test_data_set)
df = data_frame_generator(test_index,classes_test)
df.plot.bar( title = "ORIGNAL" );
print("FINAL IMAGE COUNT:")
print("Total Training Images = ",len(test_index))
plt.savefig('test_data.png', bbox_inches='tight')

### Loading images for Phase - 1 testing

In [ ]:
# print("fnames",fnames_test)
densenet_x, densenet_y = con_data(fnames_test,classes_test, DIM , index=test_index , bboxes = bboxes_test)
densenet_x = tf.keras.applications.densenet.preprocess_input(densenet_x)
desnenet_y = labels_updater(densenet_y)
densenet_x.shape , densenet_y.shape

### Loading Phase-1 Trained Model

In [ ]:
phase1_model = tf.keras.models.load_model("/home/pramod/COVID-19_Detection/models/phase-1.h5")
densenet = phase1_model.evaluate(densenet_x, densenet_y)

### Predicting NON-COVID and COVID-19

In [ ]:
densenet_preds = phase1_model.predict(densenet_x)
densenet_preds = np.rint(densenet_preds).astype("int")
densenet_preds= densenet_preds.reshape(test_data_set)

## CONFUSION MATRIX of PHASE -1 

In [ ]:
cm1 = confusion_matrix(densenet_y ,densenet_preds)
c_matrix(cm1,["Non-COVID","COVID-19"] )
plt.savefig('c_mtx.png', bbox_inches='tight')